Author: Alex Dawson-Elli  
Class: 524 Optimization  
Due: 2/9/2017

#### Problem 1) polyhedron modeling 
Find the A matrix and b vector to satisfy $Ax\leq b$ for shape (a) and shape (b) these solutions were found by inspection 

##### (a) The cube
A cube has 6 faces, so we know that $A \in \mathbb{R}^{6x3} $ Taking z to be up, y to the right and x out, the equation for the a plane in three dimensions is given as:

$$
a(x - x_0) + b(y - y_0) + c(z - z_0) = 0 
$$

Where $p_0 = [x_0,y_0,z_0]$ is a point on the plane and the normal vector to the plane is given as $ \vec{n} = [a,b,c]$.  solving this equation right face:

$$
\vec{n} = [0,1,0]  
$$

$$
a(x_1 - x_0) + b(y_1 - 1) + c(z_1 - z_0) = 0 
$$

$$
a(x_1 - x_0) + b(y_1 - 1) + c(z_1 - z_0) = 0  
$$





#### Problem 2) standard form with equality constraints:
convert the optimization problem from it's existing form into the standard form:  

$$
\text{minimize     } c^Tx 
$$

$$
\text{subject to:  }  Ax = b
$$

$$
x \geq 0
$$





##### b) solve both versions of the LP using JuMP and compare results



In [ ]:
A = 
b = 
c = 

In [ ]:
using JuMP, Clp

m = Model(solver = ClpSolver())

@variable(m,x[1:10] >= 0)       
@constraint(m, -z₁ + 6z₂ - z₃ + z₄ >= -3)
@constraint(m,       7z₂      + z₄ >=  5)
@constraint(m,           + z₃ + z₄ <=  2)               
@objective(m, Max, 3z₁ - z₂)          


In [252]:


m = Model(solver = ClpSolver())

@variable(m, z₁)       
@variable(m, -1 <= z₂ <= 5)          
@variable(m, -1 <= z₃ <= 5)
@variable(m, -2 <= z₄ <= 2) 
@constraint(m, -z₁ + 6z₂ - z₃ + z₄ >= -3)
@constraint(m,       7z₂      + z₄ >=  5)
@constraint(m,           + z₃ + z₄ <=  2)               
@objective(m, Max, 3z₁ - z₂)          

status = solve(m)

println(status)
println("Objective value is: ", getobjectivevalue(m))

Optimal
Objective value is: 103.0


[1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]

In [11]:
1//3

1//3

#### problem 3) Alloy blending

In [251]:
#define model
m = Model(solver = ClpSolver())

#define our system matricies
composition = [2.5  0.0  1.3;
               3.0  0.0  0.8;
               0.0  0.3  0.0;
               0.0  90   0.0;
               0.0  96   4.0;
               0.0  0.4  1.2;
               0.0  0.6  0.0 ]  * .01   #handle conversion from %

avail = [400,300,600,500,200,300,250]
cost  = [200,250,150,220,240,200,165]
minGrd = .01*[2.0,0.4,1.2]' #Minimum Grade
maxGrd = .01*[3.0,0.6,1.65]' #Max Grade


using JuMP, Clp

m = Model(solver=ClpSolver())
@variable(m, rawMaterials[1:7] >= 0 )  
@constraint(m, dot(rawMaterials,ones(7)) >= 500) 
@constraint(m, rawMaterials' .<= avail')                            
@constraint(m, minGrd * dot(rawMaterials,ones(7)) .<= rawMaterials'*composition) 
@constraint(m, rawMaterials'*composition .<= maxGrd * dot(rawMaterials,ones(7)))
@objective(m, Min, dot(rawMaterials,cost) )                                      

status = solve(m)
rm = []
#println(m)
for i = 1:7
    println("p = ", getvalue(rawMaterials[i]) )
    push!(rm,getvalue(rawMaterials[i]))
end
# println(status)
# println()
#println("p = ", getvalue() )
#println("q = ", getvalue() )
 println("objective = ", getobjectivevalue(m) )



p = 400.0
p = 0.0
p = 39.77630199231041
p = 0.0
p = 2.7612722824187346
p = 57.46242572527083
p = 0.0
objective = 98121.63579168123


In [168]:
rawMaterials'

1×7 RowVector{JuMP.Variable,ConjArray{JuMP.Variable,1,Array{JuMP.Variable,1}}}:
 rawMaterials[1]  rawMaterials[2]  …  rawMaterials[6]  rawMaterials[7]

In [167]:
rawMaterials'*composition

1×3 RowVector{JuMP.GenericAffExpr{Float64,JuMP.Variable},ConjArray{JuMP.GenericAffExpr{Float64,JuMP.Variable},1,Array{JuMP.GenericAffExpr{Float64,JuMP.Variable},1}}}:
 0.025 rawMaterials[1] + 0.03 rawMaterials[2]  …  0.013000000000000001 rawMaterials[1] + 0.008 rawMaterials[2] + 0.04 rawMaterials[5] + 0.012 rawMaterials[6]

In [28]:
minGrd = [2.0,0.4,1.2]*.01' 

3-element Array{Float64,1}:
 0.02 
 0.004
 0.012

In [107]:
rm'*composition / 500

1×3 RowVector{Float64,Array{Float64,1}}:
 0.02  0.006  0.012

In [99]:
PS'*composition / 500

1×3 RowVector{Float64,Array{Float64,1}}:
 0.0125  0.003  0.0065

#### problem 4) stingler's diet  

##### a) reproduce findings

Lets begin by importing the data from the CSV file - using the provided code. I've modified this code slightly to make implementing matrix constraints easier, as the notation is quite compact. 




In [200]:
# import Stigler's data set
raw = readcsv("stigler.csv")
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lconst = convert(Array{Float64},raw[2,n_nutrients])  #named arrays are a bit verbose

# data[f,i] is the amount of nutrient i contained in food f.
dataMat = raw[n_foods,n_nutrients]  #named arrays are a bit verbose
dataMat = convert(Array{Float64},dataMat); #make it nice and floaty

In [211]:
using JuMP, Clp

m = Model(solver=ClpSolver())

@variable(m, Foods[1:size(foods)[1]] >= 0 )  
@constraint(m, Foods'*dataMat  .>= lconst')
@objective(m, Min, dot(Foods,ones(size(foods)[1])))                                      

status = solve(m)


:Optimal

Cool, our solver completed sucessfully. Lets check quickly that our constraints our met

In [245]:
foodsChoosen = []
for i = 1:77
    push!(foodsChoosen,getvalue(Foods[i]))    
end
println(foodsChoosen'*dataMat)
println(lconst')
differ = (convert(Array{Float64},foodsChoosen'*dataMat)[1,:] - lconst)'
println("diff = ", differ)

Any[3.0 147.414 0.8 60.4669 5.0 4.12044 2.7 27.316 75.0]
[3.0 70.0 0.8 12.0 5.0 1.8 2.7 18.0 75.0]
diff = [0.0 77.4135 1.11022e-16 48.4669 0.0 2.32044 0.0 9.31598 -1.42109e-14]


so we see our constraints are met. let's see how stigler did without a computer.

In [220]:
println("cost of food over a year is: ", getobjectivevalue(m)*365 )

cost of food over a year is: 39.66173154546625


Stigler didn't do too bad.

##### b) repeat with vegetarian diet

I've manually filtered the stigler list to only include vegetarian cuizine because let's face it, who has time to figure out how to pop a row from a Named Array in code. below, we reload the stiglerVeg.csv and repeat the above analysis


In [247]:
# import Stigler's data set
raw = readcsv("stiglerVeg.csv")
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lconst = convert(Array{Float64},raw[2,n_nutrients])  #named arrays are a bit verbose

# data[f,i] is the amount of nutrient i contained in food f.
dataMat = raw[n_foods,n_nutrients]  #named arrays are a bit verbose
dataMat = convert(Array{Float64},dataMat); #make it nice and floaty

In [249]:
using JuMP, Clp

m = Model(solver=ClpSolver())

@variable(m, Foods[1:size(foods)[1]] >= 0 )  
@constraint(m, Foods'*dataMat  .>= lconst')
@objective(m, Min, dot(Foods,ones(size(foods)[1])))                                      

status = solve(m)


:Optimal

In [250]:
println("cost of food over a year is: ", getobjectivevalue(m)*365 )

cost of food over a year is: 39.79866435040896


The cost of food as a vegetarian is higher, but not by much